# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   605.563100 -0.484493  1234  605.078607  606.047592  1210.157214   
6   415.812694  0.467391  1234  416.280085  415.345302   832.560171   
7   188.540296  0.971079  1234  189.511376  187.569217   379.022751   
8   176.965037  0.089680  1234  177.054717  176.875357   354.109435   
9   151.448207  0.988494  1234  152.436701  150.459714   304.873402   
10  976.790183  0.497116  1234  977.287299  976.293067  1954.574598   
11  600.892989  0.057112  1234  600.950101  600.835877  1201.900201   
12  489.189119 -0.873362  1234  488.315757  490.062481   976.631514   
13  714.483073  0.473333  1234  714.956406  714.009740  1429.912813   
14  171.005031 -0.310930  1234  170.694101  171.315961   341.388202   

              f  
5   1212.095184  
6    830.690605  
7    375.138434  
8    353.750714  
9    300.919428  
10  1952.586135  
11  1201.671754  
12   980.124962  
13  1428.019479  
14   342.631922

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -605.078607   606.047592
6   -416.280085   415.345302
7   -189.511376   187.569217
8   -177.054717   176.875357
9   -152.436701   150.459714
10  -977.287299   976.293067
11  -600.950101   600.835877
12  -488.315757   490.062481
13  -714.956406   714.009740
14  -170.694101   171.315961

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -605.078607   606.047592
6   -416.280085   415.345302
7   -189.511376   187.569217
8   -177.054717   176.875357
9   -152.436701   150.459714
10  -977.287299   976.293067
11  -600.950101   600.835877
12  -488.315757   490.062481
13  -714.956406   714.009740
14  -170.694101   171.315961

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-605.07860698,  606.0475922 ],
       [-416.2800853 ,  415.34530232],
       [-189.51137566,  187.56921689],
       [-177.05471747,  176.87535696],
       [-152.43670105,  150.4597139 ],
       [-977.28729918,  976.29306726],
       [-600.95010062,  600.83587715],
       [-488.31575697,  490.06248099],
       [-714.95640638,  714.00973971],
       [-170.69410115,  171.3159608 ]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1208.157214  -1212.095184
6     830.560171   -830.690605
7     377.022751   -375.138434
8     352.109435   -353.750714
9     302.873402   -300.919428
10   1952.574598  -1952.586135
11   1199.900201  -1201.671754
12    974.631514   -980.124962
13   1427.912813  -1428.019479
14    339.388202   -342.631922

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf